In [1]:
#!pip3 install "gym[atari]"
import gym
import sys
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import  pickle
import torch

In [2]:
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")
query_environment("ALE/Tetris-v5")

Action Space: Discrete(5)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: 27000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [3]:
def encode_simple(X):
  #как сделать простое кодирование? Ну давай разобьём картинку двумя разными сетками и измерим яркость
  #на входе одиночная картинка
  y_lst = []
  sz = np.shape(X)
  count_squares = 12
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 9
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 5
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  #count_squares = 2
  #for i in range(count_squares):
  #  for j in range(count_squares):
  #    x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
  #    y_lst.append(np.nanmean(x))
  #размер: 62
  return np.array(y_lst)

In [4]:
n_actions = 5
embed_size = 250#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(input_size=embed_size,output_size=n_actions)
try:
    with open('./genoms/best_genom_tetris.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]
    print('loaded successfully')
except Exception:
    genom = nt.nn.disassemble_genom()

loaded successfully


In [5]:
#3, 1, 4, 4

In [6]:
n_actions = 5
embed_size = 100800#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(tacts=1,input_size=embed_size,output_size=n_actions)
genom = nt.nn.disassemble_genom()

In [7]:
def test_atary(genom, draw=False,game_name="ALE/Tetris-v5",seed=0,controller=None):

  env = gym.make(game_name)
  env.reset()
  n_actions = env.action_space.n
  state_dim = env.observation_space.shape#здесь картинка
  #print('n_actions',n_actions,'state_dim',state_dim)
  #1/0
  state_dim = embed_size#это эмбеддинг картинки
  if controller==None: controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  out_tape = np.zeros(30)
  reward_sum = 0

  #seed=1
  #np.random.seed(seed)
  #env.seed(seed)
  while True:
  #for i in range(200):
      action = np.ravel(out_tape)
      action += np.random.rand(len(action))*np.std(action)*0.05#игра детерминистическая, иначе рандома не будет
      #print('action',int(np.argmax(action)),action)
      #исполнить env
      state, reward, done,_ = env.step(int(np.argmax(action)))
        
      shp = np.shape(state)
      #print(shp)
      #1/0
      state = np.reshape(state,[shp[2],shp[0],shp[1]])
      #print(np.shape(state))
        
      t=pd.Timestamp.now()
      if done:
        break
      #state = encode_mobnet(np.array([state]))
      #state = encode_simple(state)
      reward_sum += (reward + 0.1)
      out_tape = controller.act(torch.tensor(state),reward,done)
      if draw:
        globals()['video'].append(Image.fromarray(env.render("rgb_array")))
  return reward_sum, controller

In [8]:
def multy_test(genom,draw=False):
  if draw:
    globals()['video'] = []

  game_name = "ALE/Tetris-v5"
  q_arr = []
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=100800,output_size=n_actions) 
  for i in range(3):
    q,controller = test_atary(genom, draw=draw,game_name=game_name,controller=controller)
    q_arr.append(q*(i+1))
  return np.sum(q_arr)/len(q_arr)

In [9]:
#(210, 160, 3)

In [10]:
%%time
q,_=test_atary(genom,game_name="ALE/Tetris-v5", draw=False)
print('q=',q)

q= 41.70000000000032
CPU times: user 3.27 s, sys: 7.58 ms, total: 3.28 s
Wall time: 630 ms


In [11]:
%%time
q=multy_test(genom)
print('q=',q)
#q=multy_test(genom)
#print('q=',q)

q= 83.40000000000065
CPU times: user 10.2 s, sys: 5.88 ms, total: 10.2 s
Wall time: 1.93 s


In [12]:
%%time
q=test_atary(genom)
print('q=',q)

q= (41.70000000000032, <neural_tape_controller.nt_controller object at 0x7f2f038451f0>)
CPU times: user 3.26 s, sys: 3.27 ms, total: 3.26 s
Wall time: 627 ms


In [13]:
print(pd.Timestamp.now())

2022-06-08 19:56:26.443397


In [14]:
print(1)
opt = optimize.optimizer(multy_test, genom_size=len(genom),parallel_cores=1,init_file='genoms/genom1.pkl',history_file='history/history.pkl')

1
loaded successfully
history loaded successfully


In [1]:
try:
    with open('genoms/best_genom_tetris.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

In [15]:

opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open('genoms/best_genom_tetris.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

opt# 0
scores for optimizers augmented tensor([-4.6445e+01,  1.0000e+10,  1.0000e+10,  1.0000e+10, -4.0000e-02,
         1.0000e+10,  1.0000e+10,  1.0000e+10,  1.0000e+10])
chosen gradient_wide_50 previous_result: nan per tacts: nan
score_new 112.20000000000105 score_prev 109.00000000000101 gained 3.2000000000000455
score_new 111.13333333333436 score_prev 112.20000000000105 gained -1.0666666666666913
but success with one derivative: score_new 95.80000000000082 score_prev 112.20000000000105 gained -16.400000000000233
undo
score_new 109.00000000000101 score_prev 112.20000000000105 gained -3.2000000000000455
but success with one derivative: score_new 140.06666666666658 score_prev 112.20000000000105 gained 27.866666666665523
score_new 108.600000000001 score_prev 140.06666666666658 gained -31.466666666665574
but success with one derivative: score_new 109.00000000000101 score_prev 140.06666666666658 gained -31.06666666666557
undo
score_new 115.26666666666775 score_prev 140.06666666666658 gai

In [ ]:
with open('./genoms/genom1.pkl', 'rb') as f:
        genom = pickle.load(f)
        #genom = genom[-1]

In [ ]:
q=multy_test(genom,draw=True)
print('q=',q)

q= 90.86666666666578


In [ ]:
video[0].save(
    './out_videos/tetris.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=100,
    loop=0
)

In [ ]:
#noise stategy
#with open('genoms/best_genom_tetris.pkl', 'rb') as f:
    #genom = pickle.load(f)[-1]
globals()['video'] = []
noise = np.random.normal(size=[len(genom)])*1
for i in range(1):
    genom = genom+noise
    q=test_atary(genom, draw=True,game_name="ALE/Tetris-v5",seed=0)
    np.random.seed(i)
    noise = np.random.normal(size=[len(genom)])*1



In [ ]:
video[0].save(
    './out_videos/tetris.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=10,
    loop=0
)

In [ ]:
with open('genoms/best_genom_tetris.pkl', 'wb') as f:
    pickle.dump(genom,f,protocol=pickle.HIGHEST_PROTOCOL)
    print('WRITTEN')